# SLOWLY ROTATING CASE IN ENTANGLED RELATIVITY FOR ELECTRICALLY CHARGED BLACK HOLES

Entangled relativity (ER) is based on the idea that there can be no space-time without matter, a hypothesis that aligns with and encompasses Mach's principle. Therefore, this new theory serves as an extension of general relativity (GR), providing its predictions in the presence of weak fields. The purpose of this notebook is to test solutions for slowly rotating black holes within the framework of entangled relativity and demonstrate their validity. 

The ER action reads:

$S=-\frac{\xi}{2c}\int d^{4}x\sqrt{-g}\frac{\mathcal{L}_{m}^{2}}{R}$ where $\xi$ has the dimension of the usual coupling constant of GR $\kappa=\frac{8\pi G}{c^{4}}$ and $\mathcal{L}_{m}$ is a scalar Lagrangian representing the matter fields. The corresponding equations of motion are:

\begin{eqnarray*}
-\frac{R}{\mathcal{L}_{m}}T_{\mu\nu}+\frac{R^{2}}{\mathcal{L}_{m}^{2}}\left(\nabla_{\mu}\nabla_{\nu} - g_{\mu\nu}\square\right)  \frac{\mathcal{L}_{m}^{2}}{R^{2}} &=& R_{\mu\nu}-\frac{1}{2}g_{\mu\nu}R\\
\end{eqnarray*}

This action also has a scalar-tensor form: $\frac{1}{c}\frac{\xi}{\tilde{k}}\int d^{4}x\sqrt{-g}\left[\frac{\phi R}{2\tilde{k}}+\sqrt{\phi}\mathcal{L}_{m}\right]$ with the motion equations:

\begin{eqnarray}
G_{\mu\nu} = \tilde{k} \frac{T_{\alpha\beta}}{\sqrt{\phi}}+\frac{1}{\phi}\left[\nabla_{\mu}\nabla_{\nu} - g_{\mu\nu}\square\right]\phi\hspace{2cm}(1)
\end{eqnarray} 

where $\sqrt{\phi}=-\tilde{\kappa}\mathcal{L}_{m}/R$ and the stress-energy tensor $T_{\mu\nu}=-\frac{2}{\sqrt{-g}}\frac{\partial(\sqrt{-g}\mathcal{L}_{m})}{\partial g^{\mu\nu}}$. It can be shown that this scalar-tensor form expressed in the Einstein frame is a special case of a dilaton theory. 

In the case of slowly rotating black holes, the work of J. H. Horne and T. Horowitz (1992) - Physical Review D provides solutions for arbitrary coupling of the dilaton with the Maxwell field (governed by the free parameter $\alpha$) that we wish to test here. Knowing that ER corresponds to the particular case of $\alpha=\frac{1}{2\sqrt{3}}$, the metric and the various quantities that come into play (see notebook SLOWLY ROTATING BLACK HOLES) are now written as follows:

$\Psi=\left[e^{-2\alpha\phi}\right]_{|_{\alpha=(2\sqrt{3})^{-1}}}=\left[\left(1-\frac{r_{-}}{r}\right)^{\frac{2\alpha^{2}}{1+\alpha^{2}}}\right]_{|_{\alpha=(2\sqrt{3})^{-1}}}=\left(1-\frac{r_{-}}{r}\right)^{\frac{-2}{13}}$ and $\Phi=\sqrt{3}\ln{(\Psi)}=\frac{2\sqrt{3}}{13}\ln{(1-\frac{r_{m}}{r})}$

The metric becomes:

$ds^{2}=-\lambda^{2}dt^{2}+\frac{dr^{2}}{\lambda^{2}}+R^{2}d\Omega-2af(r)\sin{\theta}^{2}dtd\phi$

where $\lambda^{2}=\left(1-\frac{r_{+}}{r}\right)\left(1-\frac{r_{-}}{r}\right)^{\frac{11}{13}}$ and $R^{2}=r^{2}\left(1-\frac{r_{-}}{r}\right)^{\frac{2}{13}}$. The non diagonal term is driven by the function $f(r)$:

$f(r)=\frac{169 r^2 (1-\frac{r_{-}}{r})^{\frac{2}{13}}}{99 r_{-}^2} - (1-\frac{r_{-}}{r})^{\frac{11}{13}} \left( 1 + \frac{169 r^2}{99 r_{-}^2} + \frac{13r}{11 r_{-}} - \frac{r_{+}}{r} \right)
$


Note also that $A_{t}=\frac{Q}{r}$, $A_{\phi}=-a\sin^2{\theta}\frac{Q}{r}$ and $Q=\left(\frac{12 r_{+}r_{-}}{13}\right)$

The complexity of some calculations in this notebook prompted us to perform intermediate first order Taylor expansions to maintain tractability of the different operations using the Sagemath language. This does not affect the validity of our results since they are part of a first-order analysis in 'a'.

In [30]:
version()

'SageMath version 10.1, Release Date: 2023-08-20'

In [31]:
%display latex

'SageMath version used is 10.1, Release Date: 2023-08-20'

In [32]:
from sage.manifolds.operators import dalembertian
from sage.manifolds.operators import grad

Let's write a function to replace all the different expressions and perform the taylor expansions 

In [33]:
def slow_rot(arg):
    subs_funcs = [(Z, E), (U, A), (V, B), (X, C), (Y, D)]
    final_subs = {Q: H}
    
    if hasattr(arg, 'expr'):
        arg = arg.expr()
        
    if hasattr(arg, 'apply_map')*hasattr(arg, 'display'):
        arg.apply_map(lambda f: f.taylor(a, 0, 1))
        for i, (old_func, new_func) in enumerate(subs_funcs):
            arg.apply_map(lambda f: (f.substitute_function(old_func, new_func).taylor(a, 0, 1) if i == 0 \
                                           else f.substitute_function(old_func, new_func)))                        
        arg.apply_map(lambda f: f.subs(final_subs).factor())
    else:
        arg = arg.taylor(a, 0, 1)
        for i, (old_func, new_func) in enumerate(subs_funcs):
            arg = (arg.substitute_function(old_func, new_func).taylor(a, 0, 1) if i == 0 \
                                           else arg.substitute_function(old_func, new_func)) 
        arg = arg.subs(final_subs).factor()
    
    return arg

# I. Definition of the metric in ER

In [34]:
M = Manifold(4, 'M', structure='Lorentzian')
print(M)

4-dimensional Lorentzian manifold M


In [35]:
X.<t,r,th,ph> = M.chart(r't r:(0,+oo) th:(0,pi):\theta ph:(0,2*pi):\phi')
X

Chart (M, (t, r, th, ph))

In [36]:
g = M.metric()
r_m, r_p, a, Q = var('r_m r_p a Q')

In [37]:
U = function('U')
V = function('V')
X = function('X')
Y = function('Y')
Z = function('Z')
g[0,0] = U(r)
g[0,3] = Z(r,th) 
g[1,1] = V(r)
g[2,2] = X(r)
g[3,3] = Y(r,th)
g[3,0] = Z(r,th) 

In [38]:
Psi(r) = (1-r_p/r)
Phi(r) = (1-r_m/r)
A(r) = -Psi(r)*Phi(r)**(15/13)
B(r) = 1/(Psi(r)*Phi(r)**(7/13))
C(r) = r**2*Phi(r)**(6/13)
D(r) = (r*sin(th))**2*Phi(r)**(6/13)
f(r)=(r**2*(1+(1/(2*(3)**(1/2)))**2)**2)*Phi(r)**(2*(1/(2*(3)**(1/2)))**2/(1+(1/(2*(3)**(1/2)))**2))/(((1-(1/(2*(3)**(1/2)))**2)*(1-3*(1/(2*(3)**(1/2)))**2)*r_m**2))-Phi(r)**((1-(1/(2*(3)**(1/2)))**2)/(1+(1/(2*(3)**(1/2)))**2))*(1+(1+(1/(2*(3)**(1/2)))**2)**2*r**2/((1-(1/(2*(3)**(1/2)))**2)*(1-3*(1/(2*(3)**(1/2)))**2)*r_m**2)+(1+(1/(2*(3)**(1/2)))**2)*r/((1-(1/(2*(3)**(1/2)))**2)*r_m)-r_p/r)
E(r,th)=-a*sin(th)**2*f(r)*Phi(r)**(4/13)
H=(r_m*r_p/(1+(1/(2*(3)**(1/2)))**2))**(1/2)

In [39]:
h = g.copy()
h.set_name('h')
slow_rot(h).display()

h = -(r - r_p)*((r - r_m)/r)^(15/13)/r dt⊗dt + 1/99*(169*r^3*((r - r_m)/r)^(9/13) + 117*r^2*r_m*((r - r_m)/r)^(9/13) + 99*r*r_m^2*((r - r_m)/r)^(9/13) - 99*r_m^2*r_p*((r - r_m)/r)^(9/13) - 169*r^3)*a*((r - r_m)/r)^(6/13)*sin(th)^2/(r*r_m^2) dt⊗dph + r/((r - r_p)*((r - r_m)/r)^(7/13)) dr⊗dr + r^2*((r - r_m)/r)^(6/13) dth⊗dth + 1/99*(169*r^3*((r - r_m)/r)^(9/13) + 117*r^2*r_m*((r - r_m)/r)^(9/13) + 99*r*r_m^2*((r - r_m)/r)^(9/13) - 99*r_m^2*r_p*((r - r_m)/r)^(9/13) - 169*r^3)*a*((r - r_m)/r)^(6/13)*sin(th)^2/(r*r_m^2) dph⊗dt + r^2*((r - r_m)/r)^(6/13)*sin(th)^2 dph⊗dph

# II. The vector potential

Let's define the covariant derivative

In [40]:
nab = g.connection() 

and the potential vector field corresponding to a slowly rotating BH

In [41]:
pot_vec = M.tensor_field(0,1,name='A')
Q=var('Q')
pot_vec[0]=Q/r
pot_vec[1]=0
pot_vec[2]=0
pot_vec[3]=-a*sin(th)**2*(Q/r)
pot_vec.display()

A = Q/r dt - Q*a*sin(th)^2/r dph

## II.1. Definition of the EM tensor

We now derive the EM tensor:

$F_{\mu\nu}=\partial_{\mu}A_{\nu}-\partial_{\nu}A_{\mu}$

It can be obtained directly as the exterior derivative of the potential vector $A$ via the function `diff`
(or the method `exterior_derivative`)

In [42]:
F = diff(pot_vec)
F.set_name('F')
Fuu = F.up(g)
F.display()

F = Q/r^2 dt∧dr + Q*a*sin(th)^2/r^2 dr∧dph - 2*Q*a*cos(th)*sin(th)/r dth∧dph

The matter Lagrangian reads

In [43]:
Lm = -F['_ij']*F.up(g)['^ij']/2
show(LatexExpr(r'\mathcal{L}_{m} = '), slow_rot(Lm), LatexExpr(r'+ \mathcal{O}(a^{2})'))

\mathcal{L}_{m} =  12/13*r_m*r_p/(r^4*((r - r_m)/r)^(8/13)) + \mathcal{O}(a^{2})

# III. The motion equation  equation (1) 

## III.1. The left hand side of the motion equation (1) : the Einstein tensor

In [44]:
g.display()

g = U(r) dt⊗dt + Z(r, th) dt⊗dph + V(r) dr⊗dr + X(r) dth⊗dth + Z(r, th) dph⊗dt + Y(r, th) dph⊗dph

In [45]:
ER_ricc = g.ricci()
slow_rot(ER_ricc)

Field of symmetric bilinear forms Ric(g) on the 4-dimensional Lorentzian manifold M

In [46]:
gu = g.up(g)
slow_rot(gu)

Tensor field of type (2,0) on the 4-dimensional Lorentzian manifold M

In [47]:
ER_rsc = gu['^ab']*ER_ricc['_ab']
show(LatexExpr(r'R = '), slow_rot(ER_rsc), LatexExpr(r'+ \mathcal{O}(a^{2})'))

R =  -12/13*r_m*r_p/((r - r_m)^(6/13)*r^(46/13)) + \mathcal{O}(a^{2})

We deduce the Einstein tensor

In [48]:
G = ER_ricc - 1/2*g*ER_rsc
G.apply_map(lambda f: f.taylor(a,0,1)) 
G.apply_map(lambda f: f.substitute_function(Z,E))
G.apply_map(lambda f: f.taylor(a,0,1))
G.apply_map(lambda f: f.substitute_function(U, A).substitute_function(V, B).substitute_function(X, C).substitute_function(Y, D))
G.apply_map(lambda f: f.subs(Q=H))
G.apply_map(lambda f: f.factor())

In [49]:
G = ER_ricc - 1/2*g*ER_rsc
G.set_name(r'G_{\mu\nu}')
slow_rot(G)

Field of symmetric bilinear forms G_{\mu\nu} on the 4-dimensional Lorentzian manifold M

## III.2. The right hand side of the motion equation (1) 

### III.2.1 The first component: the stress-energy tensor $T_{\mu\nu}=2\left(F_{\rho\mu}F^{\rho}_{\hspace{0.2cm}\nu}-\frac{1}{4}g_{\mu\nu}F^{2}\right)$ 

In [50]:
Fuu = F.up(g)
F2 = F['_ab']*Fuu['^ab']
F2t = F2.expr().taylor(a,0,1)
show(LatexExpr(r'F^{2} = '), F2t)

F^{2} =  -2*Q^2/((r - r_m)^(8/13)*r^(44/13))

In [51]:
Fud = F.up(g,0)
T = 2*(F['_k.']*Fud['^k_.'] - 1/4*F2 * g)
slow_rot(T)

Tensor field of type (0,2) on the 4-dimensional Lorentzian manifold M

### III.2.2. The Scalar field

The scalar field can be expressed as $\Phi=\left[\left(1-\frac{r_{-}}{r}\right)^{\frac{2\alpha^{2}}{1+\alpha^{2}}}_{|_{\alpha=(2\sqrt{3})^{-1}}}\right]^{-2}=\left(1-\frac{r_{-}}{r}\right)^{\frac{-4\alpha^{2}}{1+\alpha^{2}}}_{|_{\alpha=(2\sqrt{3})^{-1}}}=e^{-\frac{2\phi}{\sqrt{3}}}=\left(1-\frac{r_{-}}{r}\right)^{\frac{-4}{13}}$ 

In [52]:
Phi_sc = M.scalar_field((1-r_m/r)**(-2/13), name='Phi', latex_name=r'\Phi')

We deduce the final quotient and perform a first-order Taylor expansion to simplify the expression before any further calculations

In [53]:
UU=T/Phi_sc
slow_rot(UU)

Tensor field of type (0,2) on the 4-dimensional Lorentzian manifold M

### III.2.3. The second component of the motion equation (1) $\frac{1}{\Phi}(\triangledown_{\mu}\triangledown_{\nu}-g_{\mu\nu}\square)\Phi$

We now construct the d'Alembert operator

In [54]:
Dal = dalembertian(Phi_sc**2); Dal

Scalar field Box(Phi^2) on the 4-dimensional Lorentzian manifold M

In [55]:
Gp = nab(Phi_sc**2); Gp 

1-form dPhi^2 on the 4-dimensional Lorentzian manifold M

We substitute the function Z in the expression and we perform a first first-order Taylor expansion to simplify the expression before any further calculations

In [56]:
S = Phi_sc**(-2)*(nab(Gp).down(g)-g*Dal) 
S.apply_map(lambda f: f.substitute_function(Z,E))
S.apply_map(lambda f: f.taylor(a,0,1)) 

We subsitute the remaining expressions U,V,X,Y and Q to have the final form of the right hand side of the motion equation (1)

In [57]:
RHS = UU + S
RHS.apply_map(lambda f: f.taylor(a,0,1)) 
RHS.apply_map(lambda f: f.substitute_function(U, A).substitute_function(V, B).substitute_function(X, C).substitute_function(Y, D))
RHS.apply_map(lambda f: f.subs(Q=H))

# IV. Verification of the field equations

## IV.1 Verification of the $G_{\mu\nu} = \tilde{k} \frac{T_{\alpha\beta}}{\sqrt{\phi}}+\frac{1}{\phi}\left[\nabla_{\mu}\nabla_{\nu} - g_{\mu\nu}\square\right]\phi$ equation

Let's make the difference between the right hand side and the left hand side of the equation (1)

In [58]:
G == RHS

True

## IV.2. Verification of the $\triangledown\left(\sqrt{\phi}F_{\mu\nu}\right)$ equation

In [68]:
slow_rot(Fuu)
q = Fuu*Phi_sc
slow_rot(q)

Tensor field of type (2,0) on the 4-dimensional Lorentzian manifold M

In [69]:
eq = nab(q)['_i^ij']

In [70]:
latex_str = r'\triangledown\left(\sqrt{\phi}F^{\mu\nu}\right)_{el} = ' 
show(LatexExpr(latex_str), slow_rot(eq)[:], LatexExpr(r'= [0,0,0,0]'))
eq == 0

\triangledown\left(\sqrt{\phi}F^{\mu\nu}\right)_{el} =  [0,
 0,
 0,
 -4/13*sqrt(13)*sqrt(3)*(cos(th)^2 + sin(th)^2 - 1)*a*sqrt(r_m)*sqrt(r_p)/((r - r_m)^(14/13)*r^(51/13)*sin(th)^2)] = [0,0,0,0]

True

## IV.3.  Verification of the $\phi = \frac{Lm}{R}$ equation

In [72]:
Phi_sc + slow_rot(Lm / ER_rsc) == 0

True

# In conclusion the field equations $G_{\mu\nu} = \tilde{k} \frac{T_{\alpha\beta}}{\sqrt{\phi}}+\frac{1}{\phi}\left[\nabla_{\mu}\nabla_{\nu} - g_{\mu\nu}\square\right]\phi$ and  $\triangledown\left(\sqrt{\phi}T_{\mu\nu}\right)$ are perfectly verified at the first order.

# So the electrically charged solutions are solutions in ER as well for a slowly rotating black hole.